In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Import the Data and Scale it

In [144]:
from numpy import genfromtxt
data = genfromtxt('./data/heston_data.csv', delimiter=',')
X_data = data[:,1:7]
Y_data = data[:,7:8]
print(data.shape)
print(X_data[:10,:])
print(Y_data[:10,:])


(1000, 8)
[[1.28117781 1.06998061 0.18526545 0.05510498 0.6        1.        ]
 [1.21640645 1.46173507 0.89560476 0.69198049 0.68888889 1.        ]
 [1.37258257 1.52342206 0.10622916 0.89038568 0.77777778 1.        ]
 [1.52088905 1.51995222 0.53506995 0.47962976 0.86666667 1.        ]
 [1.34988903 1.14677915 0.92678788 0.40172104 0.95555556 1.        ]
 [1.84630792 1.10197268 0.97098308 0.79246595 1.04444444 1.        ]
 [1.50170888 1.57923428 0.11215282 0.72398093 1.13333333 1.        ]
 [1.0044848  1.45276113 0.43077587 0.614811   1.22222222 1.        ]
 [1.07751815 1.15721094 0.76394658 0.12668702 1.31111111 1.        ]
 [1.78408228 1.48483291 0.90667973 0.06576785 1.4        1.        ]]
[[0.38622317]
 [0.31447242]
 [0.22531723]
 [0.13294946]
 [0.05106454]
 [0.00053709]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


In [145]:
from sklearn.preprocessing import MinMaxScaler

In [146]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_data)
Y_scaled = scaler.fit_transform(Y_data)

### Create Placeholders and Constants

In [195]:
num_inputs = X_data.shape[1]
num_outputs = Y_data.shape[1]
learning_rate = 0.1
num_steps = 2000
batch_size = 20
num_neurons = 10

X = tf.placeholder(tf.float32, [None, num_inputs])
y = tf.placeholder(tf.float32, [None, num_outputs])

In [196]:
#input: X,y,batch_size
#output: X_batch,y_batch: randomly chosen rows from X,y 
def next_batch(X,y,batch_size):
    max_int = X.shape[0]
    batch_ind = np.random.randint(0,max_int,size=batch_size)
    X_batch = X[batch_ind,:]
    y_batch = y[batch_ind,:]
    
    return X_batch,y_batch

In [197]:
from tensorflow.contrib.layers import fully_connected

#Layers
hidden1 = fully_connected(X, num_neurons, activation_fn=tf.nn.elu)
hidden2 = fully_connected(hidden1, num_neurons, activation_fn=tf.nn.elu)
hidden3 = fully_connected(hidden2, num_neurons, activation_fn=tf.nn.elu)
outputs = fully_connected(hidden3, num_outputs, activation_fn=tf.nn.softmax)

In [198]:
#Loss Function
loss = tf.reduce_mean(tf.square(outputs - y))  # MSE

In [199]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [200]:
init = tf.global_variables_initializer()

In [201]:
saver = tf.train.Saver()

In [202]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch(X_scaled,Y_scaled,batch_size)
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 100 == 0:
            
            mse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tMSE:", mse)
    
    saver.save(sess, "./models/heston_nn")

0 	MSE: 0.6585946
100 	MSE: 0.637807
200 	MSE: 0.7286867
300 	MSE: 0.71342206
400 	MSE: 0.6017022
500 	MSE: 0.658522
600 	MSE: 0.71309555
700 	MSE: 0.72235954
800 	MSE: 0.71361834
900 	MSE: 0.72189206
1000 	MSE: 0.7426561
1100 	MSE: 0.72554255
1200 	MSE: 0.79046804
1300 	MSE: 0.72015953
1400 	MSE: 0.6553092
1500 	MSE: 0.6193967
1600 	MSE: 0.59389824
1700 	MSE: 0.6592845
1800 	MSE: 0.6887647
1900 	MSE: 0.705336
